In [ ]:
# 1) dcinside.com
import pandas as pd
import numpy as np

import requests
import urllib.request
from bs4 import BeautifulSoup
    
import re
import time, datetime
from tqdm import tqdm

import os, sys, glob

pages = 6935
dc_us_url = "https://gall.dcinside.com/mgallery/board/lists/?id=stockus&page="
dc_us2_url = "https://gall.dcinside.com/mgallery/board/lists/?id=tenbagger&page="
dc_ko_url = "https://gall.dcinside.com/mgallery/board/lists/?id=kospi&page="
dc_dow_url = "https://gall.dcinside.com/mgallery/board/lists/?id=dow100&page="

def dc_crawling(name, base_url, pages):
    path = "./temp"
    if not os.path.exists(path):
        os.makedirs(path)

    nrows = 50
    error_pages = []
    for page in tqdm(range(1, pages + 1)):
        url = base_url + str(page)
        try:
            res = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
            soup = BeautifulSoup(res.text, "html.parser")
        except:
            print(f"Error: {page} page\n")
            error_pages.append(page)

        data_temp = []
        try:
            for idx in range(nrows):
                i_content = soup.find_all("tr", class_="ub-content us-post")[idx]
                date = i_content.find("td", class_="gall_date")["title"][:10]
                title = i_content.find("a").text
                view = i_content.find("td", class_="gall_count").text
                like = i_content.find("td", class_="gall_recommend").text
                reply = i_content.find("span",class_="reply_num")
                if reply == None:
                    reply = "0"
                else:
                    reply = i_content.find("span",class_="reply_num").text.strip("[,]")
                data_temp.append([date, title,view,like,reply])
        except:
            print(f"Out of Index Error: {page} page\n")
        
        df = pd.DataFrame(data_temp, columns=["Date",
                                              "Title",
                                              "View",
                                              "Like",
                                              "Reply"])

        df.to_csv(f"./temp/{str(page).zfill(len(str(pages)))}.csv", encoding="utf-8")
        df = pd.DataFrame()

    # 분산파일 통합
    file_path = glob.glob(path+"/*.csv")

    data_temp = []
    for f in tqdm(file_path):
        temp = os.path.basename(f).split(".")[0]
        data_temp.extend(pd.read_csv(f, index_col=0).values.tolist())
    df = pd.DataFrame(data_temp, columns=["Date"
                                        ,"Title"
                                        ,"View"
                                        ,"Like"
                                        ,"Reply"])
    df.to_csv(f"./{name}.csv", encoding="utf-8")
    
    # 분산파일 제거
    for f in file_path:
        os.remove(f)

    return df

dc_df = dc_crawling("dc_nasdaq_test", dc_us_url, pages)

In [ ]:
# 2) investing.com crawling

import pandas as pd
import numpy as np

import re

from bs4 import BeautifulSoup
import urllib.request

import time, datetime
from tqdm import tqdm

import os, sys, glob

pages = 500

nq_100_url = "https://kr.investing.com/indices/nq-100-futures-commentary/"
nq_url = "https://kr.investing.com/indices/nasdaq-composite-commentary/"
ko_url = "https://kr.investing.com/indices/kospi-commentary/"

def investing_crawling(name, url, pages):
    ################################################################################################
    def date_reshape(time, now):
        if re.findall(r"[가-힣]", time)[0] == "분":
            t = int(re.sub(r"[가-힣\s]", "", time))
            reshaped_time = (now - datetime.timedelta(minutes=t)).strftime("%Y-%m-%d")
        
        elif re.findall(r"[가-힣]",time)[0] == "시":
            t = int(re.sub(r"[가-힣\s]", "", time))
            reshaped_time = (now - datetime.timedelta(hours=t)).strftime("%Y-%m-%d")
            
        elif re.findall(r"[가-힣]",time)[0] == "방":
            return now.strftime("%Y-%m-%d")
        
        else:
            year = time.split()[0][:-1]
            month = time.split()[1][:-1]
            day = time.split()[2][:-1]
            reshaped_time = datetime.datetime.strptime(year + "-" + month + "-" + day,"%Y-%m-%d")
        return reshaped_time
    ################################################################################################
    
    now = datetime.datetime.now()
    error_pages = []
    for page in tqdm(range(1,pages+1)):
        try:
            url = url + str(page) 
            req = urllib.request.Request(url, headers={"User-Agent":"Mozilla/5.0"})
            f = urllib.request.urlopen(req).read().decode("utf-8")
            soup = BeautifulSoup(f, "html.parser")

            nrows = len(soup.find_all('div', class_="comment_content__AvzPV"))
            data_temp = []
            
            for idx in range(nrows):
                comment = soup.find_all('div', class_="comment_content__AvzPV")[idx].text
                temp_time = soup.find_all('span', class_="comment_user-info__AWjKG")[idx].text

                c_time = date_reshape(temp_time, now)

                like = soup.find_all('button', class_="inv-button comment_user-options__aAGUk comment_like__2mSyA")[idx].text
                dislike = soup.find_all('div', class_="comment_user-options__aAGUk comment_dislike__1FF6_")[idx].text

                data_temp.append([c_time ,comment, like, dislike])
        except:
            pass
        
        df = pd.DataFrame(data_temp, columns=["Date"
                                            ,"Title"
                                            ,"Like"
                                            ,"Dislike"])
        df.to_csv(f"./temp/{str(page).zfill(len(str(pages)))}.csv", encoding="utf-8-sig")
        df = pd.DataFrame()

    # 분산파일 통합
    path = "./temp"
    file_path = glob.glob(path+"/*.csv")

    data_temp = []
    for f in tqdm(file_path):
        temp = os.path.basename(f).split(".")[0]
        data_temp.extend(pd.read_csv(f, index_col=0).values.tolist())
    df = pd.DataFrame(data_temp, columns=["Date"
                                        ,"Title"
                                        ,"Like"
                                        ,"Dislike"])
    df.to_csv(f"./dataset/comment/{name}.csv", encoding="utf-8-sig")
    
    # 분산파일 제거
    for f in file_path:
        os.remove(f)
    
    return df

In [ ]:
# 3) instiz crawling

import pandas as pd
import numpy as np

import re

import requests
import urllib.request
from bs4 import BeautifulSoup

import time, datetime
from tqdm import tqdm

import os, sys, glob

pages = 500

ins_ko_url = "https://www.instiz.net/name?category=36&k=%5B%EA%B5%AD%EB%82%B4%EC%A3%BC%EC%8B%9D%5D&nolisting=1&stype=9&page="
ins_us_url = "https://www.instiz.net/name?category=36&k=%5B%ED%95%B4%EC%99%B8%EC%A3%BC%EC%8B%9D%5D&nolisting=1&stype=9&page="

def instiz_crawling(name, url, pages):
    ################################################################################################
    def date_reshape(time):
        t = re.sub(r"[0-9]","", time)
        today = datetime.datetime.today()
        if t == ":":
            reshaped_time = today.strftime("%Y-%m-%d")
        elif t[0] == ".":
            year = today.year
            reshaped_time = str(year) + "-" + time.replace(".","-")
        else:
            reshaped_time = time.replace(".","-")
        return reshaped_time
    ################################################################################################

    error_pages = []
    for page in tqdm(range(1, pages+1)):
        url = url + str(page)
        try:
            res = requests.get(url)
            soup = BeautifulSoup(res.text, "html.parser")

            data_temp = []
            nrows = 19
            for idx in range(nrows):
                content = soup.find_all("tr")[idx+20].find_all("a")[0].text.strip()
                date = date_reshape(soup.find_all("td", class_="listno regdate")[idx].text.split()[0])
                view = soup.find_all("td", class_="listno", width=False, no=False)[2*idx].text
                rec = soup.find_all("td", class_="listno", width=False, no=False)[2*idx+1].text
                if len(soup.find_all("tr")[idx+20].find_all("a")) == 2:
                    reply = soup.find_all("tr")[idx+20].find_all("a")[1].text
                else:
                    reply = 0
                    
                data_temp.append([date, content, view, rec, reply])
        except:
            print(f"\tFatal Error: {page} page")
            error_pages.append(page)
            
        df = pd.DataFrame(data_temp, columns=["Date"
                                            ,"Title"
                                            ,"View"
                                            ,"Like"
                                            ,"Reply"])
        df.to_csv(f"./instiz_ko/{str(page).zfill(len(str(pages)))}.csv", encoding="utf-8-sig")
        df = pd.DataFrame()

    # 분산파일 통합
    path = "./temp"
    file_path = glob.glob(path+"/*.csv")

    data_temp = []
    for f in tqdm(file_path):
        temp = os.path.basename(f).split(".")[0]
        data_temp.extend(pd.read_csv(f, index_col=0).values.tolist())
    df = pd.DataFrame(data_temp, columns=["Date"
                                        ,"Title"
                                        ,"View"
                                        ,"Like"
                                        ,"Reply"])
    df.to_csv(f"./dataset/comment/{name}.csv", encoding="utf-8-sig")
    
    # 분산파일 제거
    for f in file_path:
        os.remove(f)
    
    return df

In [ ]:
# 4) theqoo crawling

import pandas as pd
import numpy as np

import re

import requests
import urllib.request
from bs4 import BeautifulSoup

import time, datetime
from tqdm import tqdm

import os, sys, glob

pages = 500

url = "https://theqoo.net/index.php?mid=stock&page="

def theqoo_crawling(name, url, pages):
    
    ################################################################################################
    def date_reshape(time):
        t = re.sub(r"[0-9]","", time)
        today = datetime.datetime.today()
        if t == ":":
            reshaped_time = today.strftime("%Y-%m-%d")
        elif t == ".":
            year = today.year
            reshaped_time = str(year) + "-" + time.replace(".","-")
        else:
            reshaped_time = time.replace(".","-")
        return reshaped_time
    ################################################################################################

    error_pages = []
    for page in tqdm(range(1, pages+1)):
        url = url + str(page)
        try:
            res = requests.get(url)
            soup = BeautifulSoup(res.text, "html.parser")
        except:
            print(f"Error page: {page} page")
            error_pages.append(page)
        
        nrows = 20
        data_temp = []
        for idx in range(10, nrows + 10):
            title = soup.find_all("td",class_="title")[idx].text.strip()
            date = date_reshape(soup.find_all("td",class_="time")[idx].text.strip())
            if soup.find_all("td",class_="m_no")[idx].text[-1] == "만":
                views = float(re.sub(r"[가-힣]","",soup.find_all("td",class_="m_no")[idx].text))*10000
            else:
                views = float(soup.find_all("td",class_="m_no")[idx].text)
            if title[-1].isdigit():
                reply = int(title[-1])
            else:
                reply = 0

            data_temp.append([date, title, views, reply])
        df = pd.DataFrame(data_temp, columns=["Date"
                                            ,"Title"
                                            ,"View"
                                            ,"Reply"])
        df["Title"] = df["Title"].apply(lambda x: x[:x.index("\n")] if "\n" in x else x)
        df.to_csv(f"./temp/{str(page).zfill(len(str(pages)))}.csv")
        df = pd.DataFrame()

    # 분산파일 통합
    path = "./temp"
    file_path = glob.glob(path + "/*.csv")

    data_temp = []
    for f in tqdm(file_path):
        temp = os.path.basename(f).split(".")[0]
        try:
            data_temp.extend(pd.read_csv(f, index_col=0).values.tolist())
        except:
            data_temp.extend(pd.read_csv(f, index_col=0, encoding="cp949").values.tolist())
    df = pd.DataFrame(data_temp, columns=["Date"
                                        ,"Title"
                                        ,"View"
                                        ,"Reply"])
    df.to_csv(f"./dataset/comment/{name}.csv", encoding="utf-8-sig")
    
    # 분산파일 제거
    for f in file_path:
        os.remove(f)
    
    return df